In [15]:
import xml.etree.ElementTree as ET
from pathlib import Path
import flickrapi
import folium
import time
import json
import os

In [16]:
def credenciais (filename="credential.json"):
    current_dir = Path.cwd() # os.path.dirname(os.path.abspath(__file__))
    credentials_path = os.path.join(current_dir, 'credentials', filename)

    with open(credentials_path, 'r') as file:
        data = json.load(file)

        chave = data["CHAVE"]
        segredo = data["SEGREDO"]

    return chave, segredo

In [18]:
api_key, secret_key = credenciais()
flickr = flickrapi.FlickrAPI(api_key, secret_key)

# if not flickr:
#     flickr.authenticate_via_browser(perms='read')
flickr.authenticate_via_browser(perms='read')

In [6]:
latitude = "-7.144126"
longitude = "-34.857480"
raio = "8"
data_inicio = "2004-04-01"
data_final = "2020-04-08"  # Uma semana depois da data de início

In [7]:
page = 1
shots = flickr.photos.search(
    has_geo="1",
    extras="geo, owner_name, date_taken",  # Adicionando extras para obter a data em que a foto foi tirada
    privacy_filter="1",
    per_page="250",  # Há um limite máximo de 250 fotos por página
    min_taken_date=data_inicio,
    max_taken_date=data_final,
    radius_units="km",
    radius=raio,
    lat=latitude,
    lon=longitude
)

In [8]:
photos = shots.find('photos').findall('photo')

In [ ]:
for photo in photos:
    photo_id = photo.attrib['id']
    owner_name = photo.attrib['ownername']
    date_taken = photo.attrib['datetaken']
    latitude = photo.attrib['latitude']
    longitude = photo.attrib['longitude']

In [ ]:
len(photos)

250

In [9]:
imeframe = 43200                           # Unixtime for 12 hours == 60seconds *60minutes *24hours; This will query the API for a certain time. Increase this number if there aren't any decent results...
one_week = 604800                           # Unixtime for one week == 60seconds *60minutes *24hours* 7days (Scope is too large to be used - only a fraction of the data that is available gets returned)
                                            # Need a Unix convertor?: http://www.unixtimestamp.com/
                                            # Alternatively you can use the built in time module of Python.

raw_file = open("raw_data.csv", "a")        # where your datapoints will be stored at
history = open("done_ids.txt", "r")         # all photo_ID's that have been added in the past.

donepre = history.readlines()               # Preventing adding the same photo twice.
history.close()

done = []
for item in donepre:
    item = item.strip()
    done.append(item)

donepids = open("done_ids.txt", "a")
print("Ready loading history.")

Ready loading history.


In [10]:
        #################THESE ARE YOUR UPPER AND LOWER LIMITS - HARDCODED IN THE SCRIPT!###############

data_inicio = 954558000            # 01/04/2000        # Bottom time limit, we shall call for all photo's that are uploaded after this timepoint.
data_final = data_inicio + one_week # 1711940400            # 01/04/2024     # Upper time limit for our small call, the while loop will keep using this untill it reaches the enddate.)

dia_atual = time.time()                   #gets the current time in unixcode.
dia_atual = int(dia_atual)

#dia_atual = 1376284800                    #overwrites curtime with a value set in the past. You can comment this line out of you wish to go from point X to now. Carefull however, as calling flickr too long on one end may cause connection termination.
#Moet nog lopen!! 29/3/2017

        ################City variables: Latitude, Longitude, radius(in KM) HARDCODED, replace according to the example and leave within quotes!##############
latitude = "-7.144126"
longitude = "-34.857480"
raio = "8"

In [ ]:
add_data = True
one_week = 604800
total_pages = 1

while add_data:

    page = 1
    startdate = str(data_inicio)
    enddate = str(data_final)
    shots = flickr.photos.search(page=str(page),
                                 has_geo="1",
                                 extras="geo, owner_name",
                                 privacy_filter="1",
                                 per_page="250",
                                 min_upload_date=startdate,
                                 max_upload_date=enddate,
                                 radius_units="km",
                                 radius=raio,
                                 lat=latitude,
                                 lon=longitude)
    
    parsed = shots.find('photos').findall('photo')
    # parsed = json.loads(shots.decode('utf-8'))          #returns a dictionary
    for key in parsed:
        part = parsed["photos"]
        total_pages =  part["pages"]

    # print(f"There are {total_pages} pages returned by flickr")
    #print data_final
    while page <= total_pages:
        shots = flickr.photos.search(page=str(page),
                                     has_geo="1",
                                     extras="geo, owner_name",
                                     privacy_filter="1",
                                     per_page="250",
                                     min_upload_date=startdate,
                                     max_upload_date=enddate,
                                     radius_units="km",
                                     radius=raio,
                                     lat=latitude,
                                     lon=longitude)
        
        parsed = shots.find('photos').findall('photo')
        # parsed = json.loads(shots.decode('utf-8'))
# """
#         for key in parsed:
#             x =  type(parsed[key])
#             if str(x) == "<type 'dict'>":   
#                 newdict = parsed[key]       
#                 for key in newdict:
#                     y =  type(newdict[key])
#                     if str(y) == "<type 'list'>":
#                         for item in newdict[key]:
#                             for key in item:
#                                 photo_id = str(item["id"].encode("utf-8"))

#                             if photo_id not in done:
#                                 done.append(photo_id)
#                                 longt = str(item["longitude"])
#                                 lat = str(item["latitude"])
#                                 user_internal_id = str(item["owner"].encode("utf-8"))
#                                 user_name = str(item["ownername"].encode("utf-8"))
#                                 visit = "https://www.flickr.com/photos/" + user_internal_id + "/" + photo_id
#                                 #print lat
#                                 #print longt

#                                 raw_file.write('"'+ photo_id + '";"' + user_internal_id + '";"' + user_name + '";"' + lat + '";"' + longt + '";"' + visit + '"\n')
#                                 donepids.write(photo_id + "\n")
#                             else:
#                                 pass
#                                 #print "double"



#         print(str(page) + " of " + str(total_pages) + " is done.")
#         page = page+1

#         #print "page UP"
#         #print "taking new data"
# """
    
    data_inicio = data_inicio + one_week
    data_final = data_final + one_week
    #print data_inicio
    print(data_final)

    if dia_atual < data_inicio:
        add_data = False

#
raw_file.close()                            #Closing the CSV file
donepids.close()                            #Closing the progress tracker file

print("Process complete")

# ext = raw_input("Press enter to terminate the program")

In [23]:
page = 1
startdate = str(data_inicio)
enddate = str(data_final)
shots = flickr.photos.search(page=str(page),
                            has_geo="1",
                            extras="geo, owner_name",
                            privacy_filter="1",
                            per_page="250",
                            min_upload_date=startdate,
                            max_upload_date=enddate,
                            radius_units="km",
                            radius=raio,
                            lat=latitude,
                            lon=longitude)

In [22]:
parsed = shots.find('photos').findall('photo')
# parsed = json.loads(shots.decode('utf-8'))          #returns a dictionary

for key in parsed:
    part = parsed["photos"]
    total_pages =  part["pages"]

print(len(parsed))

0
